# Overview of Part I

As we start our journey into Big Data Analytics, the first thing we need to do is **get the data** in the form we need for analysis!  We'll start with an overview of how to acquire and *wrangle* data.

This notebook gives us a series of examples for:

* Acquiring data from files and remote sources
* Information extraction over HTML content
* A basic "vocabulary" of operators over tables (the relational algebra)

* "Data wrangling" or integration:
  * Cleaning and filtering data, using rules and based operations
  * Linking data across dataframes or relations
  * The need for approximate match and record linking

## The Motivating Question
To illustrate the principles, we focus on the question of how old company CEOs and founders are.  The question was in part motivated by the following New York Times article:

* Founders of Successful Tech Companies Are Mostly Middle-Aged: https://www.nytimes.com/2019/08/29/business/tech-start-up-founders-nest.html?searchResultPosition=2

So let's test this hypothesis!

In [1]:
# Let's install some libraries useful for processing web data

# For string similarity
!pip3 install py_stringsimjoin

# # lxml to parse xml tree
!pip3 install lxml

     |████████████████████████████████| 1.1 MB 8.1 MB/s 
     |████████████████████████████████| 661 kB 33.7 MB/s 
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp37-cp37m-linux_x86_64.whl size=3724097 sha256=2ed9f7e76ea529f3d207e566a5cb8a4de5a4ca26c48f7fb354899fc23de225ca
  Stored in directory: /root/.cache/pip/wheels/7c/0d/a4/c0a779a4d196bed8ed41ca12be09a93e158e4fc974b59f9fd5
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linux_x86_64.whl size=2055529 sha256=ba06ee1de693452254f6863392eab242562b617e59f40920e4ad9002055071ac
  Stored in directory: /root/.cache/pip/wheels/7e/12/86/46f9378ae730550dd2f89cad50880e70d66cabedd90074f2d2
Successfully built py-stringsimjoin py-stringmatching


In [2]:
# Imports we'll use through the notebook, collected here for simplicity

# For parsing dates and being able to compare
import datetime

# For fetching remote data
import urllib
import urllib.request

# Pandas dataframes and operations
import pandas as pd

# Numpy matrix and array operations
import numpy as np

# Sqlite is a simplistic database
import sqlite3

# Approximate string matching, see 
import py_stringsimjoin as ssj
import py_stringmatching as sm

# Data visualization
import matplotlib



# Part 1: Acquiring Data

To test our hypothesis, we might want:

1. A list of companies (and, for futher details, perhaps their lines of business)
2. A list of company CEOs
3. Ages of the CEOs

We'll go through each of these using real data from the web.

### Reading Structured Data Sources

Let's start by looking up data about companies.  An example of this is at:

https://gist.githubusercontent.com/jvilledieu/c3afe5bc21da28880a30/raw/a344034b82a11433ba6f149afa47e57567d4a18f/Companies.csv

which has some nicely detailed information about companies, their categories, when they were founded, etc.  Let's load this (remote) CSV file into a dataframe.

In [3]:
data = urllib.request.urlopen(\
       'https://gist.github.com/jvilledieu/c3afe5bc21da28880a30/raw/a344034b82a11433ba6f149afa47e57567d4a18f/Companies.csv')

company_data_df = pd.read_csv(data)
    

In [4]:
# Let's write it to SQL, and read it back

conn = sqlite3.connect('local.db')

company_data_df.to_sql("companies", conn, if_exists="replace", index=False)

pd.read_sql_query('select * from companies', conn)

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,1 750 000,acquired,USA,NY,New York City,New York,1,01/06/2012,2012-06,2012-Q2,2012.0,30/06/2012,30/06/2012
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,4 000 000,operating,USA,CA,Los Angeles,Los Angeles,2,None,None,None,NaN,04/06/2010,23/09/2010
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,40 000,operating,EST,None,Tallinn,Tallinn,1,26/10/2012,2012-10,2012-Q4,2012.0,09/08/2012,09/08/2012
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|iPhone|Apps|Mobile|iOS|E-Commerce|,Electronics,1 500 000,operating,GBR,None,London,London,1,01/04/2011,2011-04,2011-Q2,2011.0,01/04/2011,01/04/2011
4,/organization/n-plusn,+n (PlusN),http://plusn.com,|Software|,Software,1 200 000,operating,USA,NY,New York City,New York,2,01/01/2012,2012-01,2012-Q1,2012.0,29/08/2012,04/09/2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47753,/organization/zzish,Zzish,http://www.zzish.com,|Analytics|Gamification|Developer APIs|iOS|Android|Education|,Education,320 000,operating,GBR,None,London,London,1,28/01/2013,2013-01,2013-Q1,2013.0,24/03/2014,24/03/2014
47754,/organization/zznode-science-and-technology-co-ltd,ZZNode Science and Technology,http://www.zznode.com,|Enterprise Software|,Enterprise Software,1 587 301,operating,CHN,None,Beijing,Beijing,1,None,None,None,NaN,01/04/2012,01/04/2012
47755,/organization/zzzzapp-com,Zzzzapp Wireless ltd.,http://www.zzzzapp.com,|Web Development|Advertising|Wireless|Mobile|,Web Development,97 398,operating,HRV,None,Split,Split,5,13/05/2012,2012-05,2012-Q2,2012.0,01/11/2011,10/09/2014
47756,/organization/a-list-games,[a]list games,http://www.alistgames.com,|Games|,Games,9 300 000,operating,None,None,None,None,1,None,None,None,NaN,21/11/2011,21/11/2011


## Companies' CEOs: a Web Table

Now we need to figure out who the CEOs are for corporations.  One place to look is Wikipedia, which has an HTML table describing the CEOs.

https://en.wikipedia.org/wiki/List_of_chief_executive_officers#List_of_CEOs

Pandas actually makes it easy to read HTML tables...

In [5]:
# Now let's read an HTML table!

company_ceos_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_chief_executive_officers#List_of_CEOs')[0]

company_ceos_df

,Company,Executive,Title,Since,Notes,Updated
0,Accenture,Julie Sweet,CEO[1],2019,"Succeeded Pierre Nanterme, died",2019-01-31
1,Aditya Birla Group,Kumar Mangalam Birla,Chairman[2],1995[2],Part of the Birla family business house in India,2018-10-01
2,Adobe Systems,Shantanu Narayen,"Chairman, president and CEO[3]",2007,Formerly with Apple,2018-10-01
3,Agenus,Garo H. Armen,"Founder, chairman, CEO[4]",1994,Founder of the Children of Armenia Fund (COAF),2018-10-01
4,Airbus,Guillaume Faury,CEO[5],2012,Succeeded Louis Gallois,2017-11-14
...,...,...,...,...,...,...
178,Wipro,Thierry Delaporte,CEO[161],2016,Previously with Tata Consultancy Services,2017-11-11
179,WWE,Vince McMahon,Chairman and CEO[162],2009,Chairman of the executive committeeChairman since 1980President and CEO from 1980 to 1993,2017-11-11
180,Yum! Brands,Greg Creed,CEO[163],2015,Previously CEO for Taco Bell,2017-11-11
181,Zillow Group,Rich Barton,CEO[164],2019,Co-founder and previously was Zillow's CEO for nearly a decade. Succeeded Spencer Rascoff.,2018-12-10


## The Problem Gets Harder... Extracting Structured Fields

So far we have companies and CEOs.  But we don't have information on how old the CEOs are!

For a solution, we're going to go back to Wikipedia -- this time looking at the web pages for the CEOs!

This involves "crawling" the CEO pages, and "scraping" the relevant content.  In other words we have to do *information extraction*.

We'll start by constructing a list of CEO web pages, from the Company CEO dataframe above.  For this, we need to take the names and do a bit of tweaking, for example adding underscores instead of spaces.

In [6]:
crawl_list = []

for executive in company_ceos_df['Executive']:
  crawl_list.append('https://en.wikipedia.org/wiki/' + executive.replace(' ', '_'))
 
crawl_list

['https://en.wikipedia.org/wiki/Julie_Sweet',
 'https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla',
 'https://en.wikipedia.org/wiki/Shantanu_Narayen',
 'https://en.wikipedia.org/wiki/Garo_H._Armen',
 'https://en.wikipedia.org/wiki/Guillaume_Faury',
 'https://en.wikipedia.org/wiki/Daniel_Zhang',
 'https://en.wikipedia.org/wiki/Jeff_Bezos',
 'https://en.wikipedia.org/wiki/Lisa_Su',
 'https://en.wikipedia.org/wiki/Stephen_Squeri',
 'https://en.wikipedia.org/wiki/Doug_Parker',
 'https://en.wikipedia.org/wiki/Joseph_R._Swedish',
 'https://en.wikipedia.org/wiki/Tim_Cook',
 'https://en.wikipedia.org/wiki/Lakshmi_Niwas_Mittal',
 'https://en.wikipedia.org/wiki/John_Stankey',
 'https://en.wikipedia.org/wiki/Charles_Woodburn',
 'https://en.wikipedia.org/wiki/Tapan_Singhel',
 'https://en.wikipedia.org/wiki/Carlos_Torres_Vila',
 'https://en.wikipedia.org/wiki/Brian_Moynihan',
 'https://en.wikipedia.org/wiki/Jes_Staley',
 'https://en.wikipedia.org/wiki/Warren_Buffett',
 'https://en.wikipedia.org/wi

In [7]:
# Use urllib.urlopen to crawl all pages in crawl_list, and store the response of the page
# in list pages 

pages = []

for url in crawl_list:
    page = url.split("/")[-1] #extract the person name at the end of the url
    print('Looking at file %s' % page)

    # An issue: some of the accent characters won't work.  We need to convert them
    # into an HTML URL.  We'll split the URL, then use "parse.quote" to change
    # the structure, then re-form the URL
    url_list = list(urllib.parse.urlsplit(url))
    url_list[2] = urllib.parse.quote(url_list[2])
    url_ascii = urllib.parse.urlunsplit(url_list)
    try:
      response = urllib.request.urlopen((url_ascii))
      #Save page and url for later use.
      pages.append(response)
    except urllib.error.URLError as e:
      print(e.reason)


Looking at file Julie_Sweet
Looking at file Kumar_Mangalam_Birla
Looking at file Shantanu_Narayen
Looking at file Garo_H._Armen
Looking at file Guillaume_Faury
Looking at file Daniel_Zhang
Looking at file Jeff_Bezos
Looking at file Lisa_Su
Looking at file Stephen_Squeri
Looking at file Doug_Parker
Looking at file Joseph_R._Swedish
Looking at file Tim_Cook
Looking at file Lakshmi_Niwas_Mittal
Looking at file John_Stankey
Looking at file Charles_Woodburn
Looking at file Tapan_Singhel
Looking at file Carlos_Torres_Vila
Looking at file Brian_Moynihan
Looking at file Jes_Staley
Looking at file Warren_Buffett
Looking at file Hubert_Joly
Looking at file Sunil_Bharti_Mittal
Looking at file Stephen_A._Schwarzman
Looking at file Andrew_Mackenzie
Looking at file Oliver_Zipse
Looking at file Dave_Calhoun
Looking at file Rich_Lesser
Looking at file Bob_Dudley
Looking at file Denise_Morrison
Looking at file Mark_Shuttleworth
Looking at file Aiman_Ezzat
Not Found
Looking at file Richard_Fairbank
Look

In [ ]:
pages

## Populating the Table with Executives

In [8]:
# Use lxml.etree.HTML(...) on the HTML content of each page to get a DOM tree that
# can be processed via XPath to extract the bday information.  Store the CEO name, 
# webpage, and the birthdate (born) in exec_df.

# We first check that the HTML content has a table of type `vcard`,
# and then extract the `bday` information.  If there is no birthdate, the datetime 
# value is NaT (not a type). 

from lxml import etree

exec_df = pd.DataFrame(columns=['name','page','born'])

for page in pages:    
    tree = etree.HTML(page.read().decode("utf-8"))  #create a DOM tree of the page
    url = page.geturl()
    bday = tree.xpath('//table[contains(@class,"vcard")]//span[@class="bday"]/text()')
    if len(bday) > 0:
        name = url[url.rfind('/')+1:] # The part of the URL after the last /
        exec_df = exec_df.append({'name': name, 'page': url,
                   'born': datetime.datetime.strptime(bday[0], '%Y-%m-%d').date()}, ignore_index=True)
    else: 
            exec_df = exec_df.append({'name': url[url.rfind('/')+1:], 'page': url
                                      , 'born': None}, ignore_index=True)
        
exec_df

,name,page,born
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,None
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla,1967-06-14
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27
3,Garo_H._Armen,https://en.wikipedia.org/wiki/Garo_H._Armen,1953-01-31
4,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22
...,...,...,...
173,Laura_J._Alber,https://en.wikipedia.org/wiki/Laura_J._Alber,None
174,Vince_McMahon,https://en.wikipedia.org/wiki/Vince_McMahon,1945-08-24
175,Greg_Creed,https://en.wikipedia.org/wiki/Greg_Creed,None
176,Rich_Barton,https://en.wikipedia.org/wiki/Rich_Barton,None


# Part 2: Data Transformation

Looking at our data to clean via *projection*...

Generally, we can extract one "narrower" table form another by using **double brackets**.

In [ ]:
# Let's take a look at the data.  Here's a way of PROJECTING the exec_df dataframe into
# a smaller table

exec_df[['name', 'born']]

,name,born
0,Julie_Sweet,None
1,Kumar_Mangalam_Birla,1967-06-14
2,Shantanu_Narayen,1963-05-27
3,Garo_H._Armen,1953-01-31
4,Guillaume_Faury,1968-02-22
...,...,...
173,Laura_J._Alber,None
174,Vince_McMahon,1945-08-24
175,Greg_Creed,None
176,Rich_Barton,None


In [9]:
# If I use single brackets, I can extract a single column as a Series.
exec_df['name']

0               Julie_Sweet
1      Kumar_Mangalam_Birla
2          Shantanu_Narayen
3             Garo_H._Armen
4           Guillaume_Faury
               ...         
173          Laura_J._Alber
174           Vince_McMahon
175              Greg_Creed
176             Rich_Barton
177               Eric_Yuan
Name: name, Length: 178, dtype: object

In [10]:
# Notice anything awry?

for person in exec_df['name']:
    print (person)

Julie_Sweet
Kumar_Mangalam_Birla
Shantanu_Narayen
Garo_H._Armen
Guillaume_Faury
Daniel_Zhang
Jeff_Bezos
Lisa_Su
Stephen_Squeri
Doug_Parker
Joseph_R._Swedish
Tim_Cook
Lakshmi_Niwas_Mittal
John_Stankey
Charles_Woodburn
Tapan_Singhel
Carlos_Torres_Vila
Brian_Moynihan
Jes_Staley
Warren_Buffett
Hubert_Joly
Sunil_Bharti_Mittal
Stephen_A._Schwarzman
Andrew_Mackenzie
Oliver_Zipse
Dave_Calhoun
Rich_Lesser
Bob_Dudley
Denise_Morrison
Mark_Shuttleworth
Richard_Fairbank
Jim_Umpleby
Evan_G._Greenberg
Chuck_Robbins
Michael_Corbat
James_Quincey
Brian_Humphries
Brian_L._Roberts
Tidjane_Thiam
Ola_K%C3%A4llenius
Michael_Dell
Ed_Bastian
Christian_Sewing
Frank_Appel
Roland_Dickey_Jr.
Edward_D._Breen
G._V._Prasad
Devin_Wenig
Andrew_Wilson
B%C3%B6rje_Ekholm
Darren_Woods
Lisa_S._Jones
Carmine_Di_Sibio
Mark_Zuckerberg
Frederick_W._Smith
Sergio_Marchionne
Abigail_Johnson
James_Michael_Lafferty
James_Hackett
Terry_Gou
Lachlan_Murdoch
Daymond_John
Phebe_Novakovic
H._Lawrence_Culp_Jr.
Mary_T._Barra
Emma_Walmsley
D

In [11]:
def to_space(x):
  return x.replace('_', ' ')

# Let's use *apply* to call a function over each element, returning a new Series
exec_df['name'].apply(to_space)

0               Julie Sweet
1      Kumar Mangalam Birla
2          Shantanu Narayen
3             Garo H. Armen
4           Guillaume Faury
               ...         
173          Laura J. Alber
174           Vince McMahon
175              Greg Creed
176             Rich Barton
177               Eric Yuan
Name: name, Length: 178, dtype: object

In [12]:
# Let's use *apply* to call a function over each element, returning a new Series
exec_df['name'].apply(lambda x: x.replace('_', ' '))

0               Julie Sweet
1      Kumar Mangalam Birla
2          Shantanu Narayen
3             Garo H. Armen
4           Guillaume Faury
               ...         
173          Laura J. Alber
174           Vince McMahon
175              Greg Creed
176             Rich Barton
177               Eric Yuan
Name: name, Length: 178, dtype: object

In [13]:
# I can also use *apply* to call a function over the rows of a dataframe
exec_df.apply(lambda x: x['name'].replace('_', ' '), axis=1)

0               Julie Sweet
1      Kumar Mangalam Birla
2          Shantanu Narayen
3             Garo H. Armen
4           Guillaume Faury
               ...         
173          Laura J. Alber
174           Vince McMahon
175              Greg Creed
176             Rich Barton
177               Eric Yuan
Length: 178, dtype: object

In [14]:
# Let's clean the name by removing underscores...
exec_df['clean_name'] = exec_df['name'].apply(lambda x: x.replace('_', ' '))

exec_df

,name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,None,Julie Sweet
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla,1967-06-14,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27,Shantanu Narayen
3,Garo_H._Armen,https://en.wikipedia.org/wiki/Garo_H._Armen,1953-01-31,Garo H. Armen
4,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22,Guillaume Faury
...,...,...,...,...
173,Laura_J._Alber,https://en.wikipedia.org/wiki/Laura_J._Alber,None,Laura J. Alber
174,Vince_McMahon,https://en.wikipedia.org/wiki/Vince_McMahon,1945-08-24,Vince McMahon
175,Greg_Creed,https://en.wikipedia.org/wiki/Greg_Creed,None,Greg Creed
176,Rich_Barton,https://en.wikipedia.org/wiki/Rich_Barton,None,Rich Barton


In [15]:
# Here's a column
exec_df['clean_name']

0               Julie Sweet
1      Kumar Mangalam Birla
2          Shantanu Narayen
3             Garo H. Armen
4           Guillaume Faury
               ...         
173          Laura J. Alber
174           Vince McMahon
175              Greg Creed
176             Rich Barton
177               Eric Yuan
Name: clean_name, Length: 178, dtype: object

In [16]:
exec_df.rename(columns={'name': 'old_name'})

,old_name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,None,Julie Sweet
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla,1967-06-14,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27,Shantanu Narayen
3,Garo_H._Armen,https://en.wikipedia.org/wiki/Garo_H._Armen,1953-01-31,Garo H. Armen
4,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22,Guillaume Faury
...,...,...,...,...
173,Laura_J._Alber,https://en.wikipedia.org/wiki/Laura_J._Alber,None,Laura J. Alber
174,Vince_McMahon,https://en.wikipedia.org/wiki/Vince_McMahon,1945-08-24,Vince McMahon
175,Greg_Creed,https://en.wikipedia.org/wiki/Greg_Creed,None,Greg Creed
176,Rich_Barton,https://en.wikipedia.org/wiki/Rich_Barton,None,Rich Barton


In [17]:
exec_df

,name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,None,Julie Sweet
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla,1967-06-14,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27,Shantanu Narayen
3,Garo_H._Armen,https://en.wikipedia.org/wiki/Garo_H._Armen,1953-01-31,Garo H. Armen
4,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22,Guillaume Faury
...,...,...,...,...
173,Laura_J._Alber,https://en.wikipedia.org/wiki/Laura_J._Alber,None,Laura J. Alber
174,Vince_McMahon,https://en.wikipedia.org/wiki/Vince_McMahon,1945-08-24,Vince McMahon
175,Greg_Creed,https://en.wikipedia.org/wiki/Greg_Creed,None,Greg Creed
176,Rich_Barton,https://en.wikipedia.org/wiki/Rich_Barton,None,Rich Barton


In [ ]:
# We can do the same via SQL.  For the example we'll save the dataframe first...

exec_df.to_sql('temp_exec', conn, if_exists="replace")
pd.read_sql_query('select name, replace(name, "_", " ") as clean_name from temp_exec', conn)

,name,clean_name
0,Julie_Sweet,Julie Sweet
1,Kumar_Mangalam_Birla,Kumar Mangalam Birla
2,Shantanu_Narayen,Shantanu Narayen
3,Garo_H._Armen,Garo H. Armen
4,Guillaume_Faury,Guillaume Faury
...,...,...
173,Laura_J._Alber,Laura J. Alber
174,Vince_McMahon,Vince McMahon
175,Greg_Creed,Greg Creed
176,Rich_Barton,Rich Barton


## Selecting a subset of the rows

In [ ]:
# Here's a column
exec_df['clean_name']

0               Julie Sweet
1      Kumar Mangalam Birla
2          Shantanu Narayen
3             Garo H. Armen
4           Guillaume Faury
               ...         
173          Laura J. Alber
174           Vince McMahon
175              Greg Creed
176             Rich Barton
177               Eric Yuan
Name: clean_name, Length: 178, dtype: object

In [18]:
# We can apply a test (predicate) to each column, returning a Series of boolean true/false values

exec_df['clean_name'] == 'Julie Sweet'

0       True
1      False
2      False
3      False
4      False
       ...  
173    False
174    False
175    False
176    False
177    False
Name: clean_name, Length: 178, dtype: bool

In [ ]:
# If we compose these, we'll get only those rows where the boolean condition was True

exec_df[exec_df['clean_name'] == 'Julie Sweet']

,name,page,born,clean_name
0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,None,Julie Sweet


In [19]:
exec_df[exec_df['clean_name'] == 'Julie Sweet'][['page']]

,page
0,https://en.wikipedia.org/wiki/Julie_Sweet


In [20]:
# And in SQL...

exec_df.to_sql('temp_exec', conn, if_exists="replace")

pd.read_sql_query('select * from temp_exec where clean_name="Julie Sweet"', conn)

,index,name,page,born,clean_name
0,0,Julie_Sweet,https://en.wikipedia.org/wiki/Julie_Sweet,None,Julie Sweet


In [21]:
pd.read_sql_query('select page from temp_exec where clean_name="Julie Sweet"', conn)

,page
0,https://en.wikipedia.org/wiki/Julie_Sweet


In [22]:
import numpy as np

exec_df.dropna(subset=['born'])

,name,page,born,clean_name
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla,1967-06-14,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27,Shantanu Narayen
3,Garo_H._Armen,https://en.wikipedia.org/wiki/Garo_H._Armen,1953-01-31,Garo H. Armen
4,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22,Guillaume Faury
5,Daniel_Zhang,https://en.wikipedia.org/wiki/Daniel_Zhang,1972-01-11,Daniel Zhang
...,...,...,...,...
168,Doug_McMillon,https://en.wikipedia.org/wiki/Doug_McMillon,1966-10-17,Doug McMillon
170,Jason_Kilar,https://en.wikipedia.org/wiki/Jason_Kilar,1971-04-26,Jason Kilar
171,Charles_Scharf,https://en.wikipedia.org/wiki/Charles_Scharf,1965-04-24,Charles Scharf
174,Vince_McMahon,https://en.wikipedia.org/wiki/Vince_McMahon,1945-08-24,Vince McMahon


In [23]:
pd.read_sql_query('select * from temp_exec where born is not null', conn).set_index('index')

,name,page,born,clean_name
index,,,,
1,Kumar_Mangalam_Birla,https://en.wikipedia.org/wiki/Kumar_Mangalam_Birla,1967-06-14,Kumar Mangalam Birla
2,Shantanu_Narayen,https://en.wikipedia.org/wiki/Shantanu_Narayen,1963-05-27,Shantanu Narayen
3,Garo_H._Armen,https://en.wikipedia.org/wiki/Garo_H._Armen,1953-01-31,Garo H. Armen
4,Guillaume_Faury,https://en.wikipedia.org/wiki/Guillaume_Faury,1968-02-22,Guillaume Faury
5,Daniel_Zhang,https://en.wikipedia.org/wiki/Daniel_Zhang,1972-01-11,Daniel Zhang
...,...,...,...,...
168,Doug_McMillon,https://en.wikipedia.org/wiki/Doug_McMillon,1966-10-17,Doug McMillon
170,Jason_Kilar,https://en.wikipedia.org/wiki/Jason_Kilar,1971-04-26,Jason Kilar
171,Charles_Scharf,https://en.wikipedia.org/wiki/Charles_Scharf,1965-04-24,Charles Scharf


## Joining Data

We start with a simple join between company_ceos_df and exec_df and persist it to the database.  We then check how many companies did not have a match on CEO name.

In [24]:
exec_df[['clean_name', 'born']]

,clean_name,born
0,Julie Sweet,None
1,Kumar Mangalam Birla,1967-06-14
2,Shantanu Narayen,1963-05-27
3,Garo H. Armen,1953-01-31
4,Guillaume Faury,1968-02-22
...,...,...
173,Laura J. Alber,None
174,Vince McMahon,1945-08-24
175,Greg Creed,None
176,Rich Barton,None


In [25]:
company_ceos_df[['Executive', 'Company']]

,Executive,Company
0,Julie Sweet,Accenture
1,Kumar Mangalam Birla,Aditya Birla Group
2,Shantanu Narayen,Adobe Systems
3,Garo H. Armen,Agenus
4,Guillaume Faury,Airbus
...,...,...
178,Thierry Delaporte,Wipro
179,Vince McMahon,WWE
180,Greg Creed,Yum! Brands
181,Rich Barton,Zillow Group


In [26]:
company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']], 
                                                left_on=['Executive'], 
                                                right_on=['clean_name'])

,Executive,Company,clean_name,born
0,Julie Sweet,Accenture,Julie Sweet,None
1,Kumar Mangalam Birla,Aditya Birla Group,Kumar Mangalam Birla,1967-06-14
2,Shantanu Narayen,Adobe Systems,Shantanu Narayen,1963-05-27
3,Garo H. Armen,Agenus,Garo H. Armen,1953-01-31
4,Guillaume Faury,Airbus,Guillaume Faury,1968-02-22
...,...,...,...,...
170,Laura J. Alber,Williams-Sonoma,Laura J. Alber,None
171,Vince McMahon,WWE,Vince McMahon,1945-08-24
172,Greg Creed,Yum! Brands,Greg Creed,None
173,Rich Barton,Zillow Group,Rich Barton,None


In [27]:
company_ceos_df.to_sql('company_ceos', conn, if_exists="replace")
exec_df.to_sql('executives', conn, if_exists="replace")
company_data_df.to_sql('company_data', conn, if_exists='replace')

pd.read_sql_query('select Executive, Company, born from company_ceos ' +\
                  'join executives on Executive=clean_name ' +\
                  'where born is not null', conn)

,Executive,Company,born
0,Kumar Mangalam Birla,Aditya Birla Group,1967-06-14
1,Shantanu Narayen,Adobe Systems,1963-05-27
2,Garo H. Armen,Agenus,1953-01-31
3,Guillaume Faury,Airbus,1968-02-22
4,Daniel Zhang,Alibaba,1972-01-11
...,...,...,...
99,Doug McMillon,Walmart,1966-10-17
100,Jason Kilar,WarnerMedia,1971-04-26
101,Charles Scharf,Wells Fargo,1965-04-24
102,Vince McMahon,WWE,1945-08-24


###Finding the misses in the join.

Note that the join above resulted in 174 rows.  However, there are more rows in company_ceos_df so we are missing some companies.  We can see which are missed using a left outer join; setting "indicator= True" allows us to see which tuples in company_ceos_df failed to find a match (left_only, e.g. row 24 and 172).

In [28]:
pd.set_option('display.max_rows', 200)
display(company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']], 
                                                left_on=['Executive'], 
                                                right_on=['clean_name'], how="left", indicator=True))



,Executive,Company,clean_name,born,_merge
0,Julie Sweet,Accenture,Julie Sweet,None,both
1,Kumar Mangalam Birla,Aditya Birla Group,Kumar Mangalam Birla,1967-06-14,both
2,Shantanu Narayen,Adobe Systems,Shantanu Narayen,1963-05-27,both
3,Garo H. Armen,Agenus,Garo H. Armen,1953-01-31,both
4,Guillaume Faury,Airbus,Guillaume Faury,1968-02-22,both
5,Daniel Zhang,Alibaba,Daniel Zhang,1972-01-11,both
6,Jeff Bezos,Amazon,Jeff Bezos,1964-01-12,both
7,Lisa Su,Advanced Micro Devices,Lisa Su,1969-11-07,both
8,Stephen Squeri,American Express,Stephen Squeri,None,both
9,Doug Parker,American Airlines Group,Doug Parker,1961-10-31,both


In [29]:
pd.set_option('display.max_rows', 50)
result_df = company_ceos_df[['Executive', 'Company']].merge(exec_df[['clean_name', 'born']], 
                                                left_on=['Executive'], 
                                                right_on=['clean_name'], how="outer", indicator=True)

result_df[result_df['_merge'] != 'both']


,Executive,Company,clean_name,born,_merge
30,Aiman Ezzat,Capgemini,NaN,NaN,left_only
40,Ola Källenius,Daimler AG,NaN,NaN,left_only
50,Börje Ekholm,Ericsson,NaN,NaN,left_only
91,Andrew Meslow,"Bath & Body Works, Inc.",NaN,NaN,left_only
109,Sapnesh Lalla,NIIT,NaN,NaN,left_only
130,Michael O'Leary,Ryanair,NaN,NaN,left_only
157,Umberto Macchi di Cellere,Tod's,NaN,NaN,left_only
178,Thierry Delaporte,Wipro,NaN,NaN,left_only
183,NaN,NaN,Ola K%C3%A4llenius,1969-06-11,right_only
184,NaN,NaN,B%C3%B6rje Ekholm,None,right_only


## Composing Joins



In [ ]:
pd.read_sql_query('select Executive, Company, born  from company_ceos ' +\
                  'join executives on Executive=clean_name join company_data cd on Company=cd.name ' +\
                  'where born is not null', conn)

,Executive,Company,born
0,Garo H. Armen,Agenus,1953-01-31
1,Daniel Zhang,Alibaba,1972-01-11
2,Jeff Bezos,Amazon,1964-01-12
3,Mark Shuttleworth,Canonical,1973-09-18
4,Brian L. Roberts,Comcast,1959-06-28
5,Mark Zuckerberg,Facebook,1984-05-14
6,Sundar Pichai,Google,1972-06-10
7,Darius Adamczyk,Honeywell,1966-02-08
8,Marillyn Hewson,Lockheed Martin,1953-12-27
9,Robert James Thomson,News Corp,1961-03-11


In [30]:
pd.read_sql_query('select * from company_ceos ' +\
                  'join executives on Executive=clean_name join company_data cd on Company=cd.name ' +\
                  'where born is not null', conn)

,index,Company,Executive,Title,Since,Notes,Updated,index,name,page,born,clean_name,index,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at
0,3,Agenus,Garo H. Armen,"Founder, chairman, CEO[4]",1994,Founder of the Children of Armenia Fund (COAF),2018-10-01,3,Garo_H._Armen,https://en.wikipedia.org/wiki/Garo_H._Armen,1953-01-31,Garo H. Armen,1442,/organization/agenus,Agenus,http://agenusbio.com,|Biotechnology|,Biotechnology,15 000 000,operating,USA,MA,Boston,Lexington,2,01/01/1994,1994-01,1994-Q1,1994.0,04/08/2009,25/04/2013
1,5,Alibaba,Daniel Zhang,CEO[6],2015,Previously with Taobao,2018-10-01,5,Daniel_Zhang,https://en.wikipedia.org/wiki/Daniel_Zhang,1972-01-11,Daniel Zhang,1808,/organization/alibaba,Alibaba,http://www.alibaba.com,|E-Commerce|,E-Commerce,1 112 000 000,operating,CHN,None,Hangzhou,Hangzhou,4,01/06/1999,1999-06,1999-Q2,1999.0,01/10/1999,11/08/2005
2,6,Amazon,Jeff Bezos,"Founder, President and chairman[7]",1994,Founder of the firm and one of the world's wealthiest people,2018-10-01,6,Jeff_Bezos,https://en.wikipedia.org/wiki/Jeff_Bezos,1964-01-12,Jeff Bezos,2100,/organization/amazon,Amazon,http://amazon.com,|Consumer Goods|Groceries|Crowdsourcing|E-Commerce|,Consumer Goods,8 000 000,operating,USA,WA,Seattle,Seattle,1,01/01/1994,1994-01,1994-Q1,1994.0,01/07/1995,01/07/1995
3,29,Canonical,Mark Shuttleworth,Founder and CEO[29],2017,Stepped down as CEO in 2009 and resumed in 2017,2017-11-14,29,Mark_Shuttleworth,https://en.wikipedia.org/wiki/Mark_Shuttleworth,1973-09-18,Mark Shuttleworth,6832,/organization/canonical-ltd,Canonical,http://www.canonical.com,|Linux|,Linux,10 000 000,operating,GBR,None,London,London,1,01/01/2004,2004-01,2004-Q1,2004.0,16/08/2013,16/08/2013
4,38,Comcast,Brian L. Roberts,Chairman and CEO[38],2002,"Son of Ralph J. Roberts, the founder of Comcast",2017-11-14,37,Brian_L._Roberts,https://en.wikipedia.org/wiki/Brian_L._Roberts,1959-06-28,Brian L. Roberts,9048,/organization/comcast,Comcast,http://comcast.com,|Web Hosting|,Web Hosting,725 000 000,operating,USA,PA,Philadelphia,Philadelphia,1,01/01/1963,1963-01,1963-Q1,1963.0,08/05/2013,08/05/2013
5,54,Facebook,Mark Zuckerberg,Chairman and CEO,2004,Co-founded Facebook in 2004,2017-11-14,53,Mark_Zuckerberg,https://en.wikipedia.org/wiki/Mark_Zuckerberg,1984-05-14,Mark Zuckerberg,14216,/organization/facebook,Facebook,http://facebook.com,|Networking|Communities|Identity|All Students|Colleges|Facebook Applications|Social Media|,Communities,2 425 700 000,operating,USA,CA,SF Bay Area,Menlo Park,11,04/02/2004,2004-02,2004-Q1,2004.0,01/09/2004,21/01/2011
6,68,Google,Sundar Pichai,CEO[60],2015,Former director of Jive Software from 2011 to 2013,2017-11-13,67,Sundar_Pichai,https://en.wikipedia.org/wiki/Sundar_Pichai,1972-06-10,Sundar Pichai,17029,/organization/google,Google,http://www.google.com,|Software|Video Streaming|Information Technology|Blogging Platforms|Email|Search|,Software,25 100 000,operating,USA,CA,SF Bay Area,Mountain View,2,07/09/1998,1998-09,1998-Q3,1998.0,01/08/1998,07/06/1999
7,74,Honeywell,Darius Adamczyk,"Chairman, President, and CEO[66]",2017,Succeeded David M. Cote (now the executive chairman),2017-11-13,73,Darius_Adamczyk,https://en.wikipedia.org/wiki/Darius_Adamczyk,1966-02-08,Darius Adamczyk,18602,/organization/honeywell,Honeywell,http://www.honeywell.com,|Technology|Manufacturing|,Manufacturing,11 400 000,operating,USA,NJ,Newark,Morristown,1,23/04/1985,1985-04,1985-Q2,1985.0,27/10/2009,27/10/2009
8,92,Lockheed Martin,Marillyn Hewson,"Chairman, president and CEO[81]",2013,Also serves on the board of Sandia National Laboratories and DuPont,2017-11-13,90,Marillyn_Hewson,https://en.wikipedia.org/wiki/Marillyn_Hewson,1953-12-27,Marillyn Hewson,23699,/organization/lockheed-martin,Lockheed Martin,http://www.lockheedmartin.com,|Design|Security|,Design,27 400 000,operating,USA,MD,"Washingto

In [31]:
company_ceos_df.merge(exec_df, 
                      left_on=['Executive'], 
                      right_on=['clean_name']).\
                      merge(company_data_df,
                            left_on='Company',
                            right_on='name')

,Company,Executive,Title,Since,Notes,Updated,name_x,page,born,clean_name,permalink,name_y,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at
0,Agenus,Garo H. Armen,"Founder, chairman, CEO[4]",1994,Founder of the Children of Armenia Fund (COAF),2018-10-01,Garo_H._Armen,https://en.wikipedia.org/wiki/Garo_H._Armen,1953-01-31,Garo H. Armen,/organization/agenus,Agenus,http://agenusbio.com,|Biotechnology|,Biotechnology,15 000 000,operating,USA,MA,Boston,Lexington,2,01/01/1994,1994-01,1994-Q1,1994.0,04/08/2009,25/04/2013
1,Alibaba,Daniel Zhang,CEO[6],2015,Previously with Taobao,2018-10-01,Daniel_Zhang,https://en.wikipedia.org/wiki/Daniel_Zhang,1972-01-11,Daniel Zhang,/organization/alibaba,Alibaba,http://www.alibaba.com,|E-Commerce|,E-Commerce,1 112 000 000,operating,CHN,NaN,Hangzhou,Hangzhou,4,01/06/1999,1999-06,1999-Q2,1999.0,01/10/1999,11/08/2005
2,Amazon,Jeff Bezos,"Founder, President and chairman[7]",1994,Founder of the firm and one of the world's wealthiest people,2018-10-01,Jeff_Bezos,https://en.wikipedia.org/wiki/Jeff_Bezos,1964-01-12,Jeff Bezos,/organization/amazon,Amazon,http://amazon.com,|Consumer Goods|Groceries|Crowdsourcing|E-Commerce|,Consumer Goods,8 000 000,operating,USA,WA,Seattle,Seattle,1,01/01/1994,1994-01,1994-Q1,1994.0,01/07/1995,01/07/1995
3,BAE Systems,Charles Woodburn,Chief Executive Officer[15],2008,Succeeded Ian King,2018-10-01,Charles_Woodburn,https://en.wikipedia.org/wiki/Charles_Woodburn,None,Charles Woodburn,/organization/bae-systems,BAE Systems,http://www.baesystems.com,|Transportation|Manufacturing|,Transportation,83 700 000,operating,USA,MD,"Washington, D.C.",Rockville,2,01/11/1999,1999-11,1999-Q4,1999.0,30/11/2009,10/09/2010
4,Canonical,Mark Shuttleworth,Founder and CEO[29],2017,Stepped down as CEO in 2009 and resumed in 2017,2017-11-14,Mark_Shuttleworth,https://en.wikipedia.org/wiki/Mark_Shuttleworth,1973-09-18,Mark Shuttleworth,/organization/canonical-ltd,Canonical,http://www.canonical.com,|Linux|,Linux,10 000 000,operating,GBR,NaN,London,London,1,01/01/2004,2004-01,2004-Q1,2004.0,16/08/2013,16/08/2013
5,Comcast,Brian L. Roberts,Chairman and CEO[38],2002,"Son of Ralph J. Roberts, the founder of Comcast",2017-11-14,Brian_L._Roberts,https://en.wikipedia.org/wiki/Brian_L._Roberts,1959-06-28,Brian L. Roberts,/organization/comcast,Comcast,http://comcast.com,|Web Hosting|,Web Hosting,725 000 000,operating,USA,PA,Philadelphia,Philadelphia,1,01/01/1963,1963-01,1963-Q1,1963.0,08/05/2013,08/05/2013
6,eBay,Devin Wenig,President and CEO[46],2015,"Third CEO of the company, after Meg Whitman and John Donahoe",2017-11-14,Devin_Wenig,https://en.wikipedia.org/wiki/Devin_Wenig,None,Devin Wenig,/organization/ebay,eBay,http://ebay.com,|Business Services|All Markets|Search|Auctions|E-Commerce|,Business Services,6 700 000,operating,USA,CA,SF Bay Area,San Jose,1,01/09/1995,1995-09,1995-Q3,1995.0,01/06/1997,01/06/1997
7,Facebook,Mark Zuckerberg,Chairman and CEO,2004,Co-founded Facebook in 2004,2017-11-14,Mark_Zuckerberg,https://en.wikipedia.org/wiki/Mark_Zuckerberg,1984-05-14,Mark Zuckerberg,/organization/facebook,Facebook,http://facebook.com,|Networking|Communities|Identity|All Students|Colleges|Facebook Applications|Social Media|,Communities,2 425 700 000,operating,USA,CA,SF Bay Area,Menlo Park,11,04/02/2004,2004-02,2004-Q1,2004.0,01/09/2004,21/01/2011
8,General Dynamics,Phebe Novakovic,Chairman and CEO[56],2014,Worked for the Central Intelligence Agency and the United States Department of Defense,2017-11-14,Phebe_Novakovic,https://en.wikipedia.org/wiki/Phebe_Novakovic,None,Phebe Novakovic,/organization/general-dynamics,General Dynamics,http://www.gd.com,|Public Transportation|,Public Transportation,84 900 000,operating,USA,VA,"Washington, D.C.",Falls Church,3,01/01/1952,1952-01,1952-Q1,1952.0,05/08/2009,10/09/2010
9,General Electric,H. Lawrence Culp Jr.,Chairman and CEO,2018

## Tasks to be submitted on Blackboard after the lab by 17.00 PM.

#### Task-1: Using Pandas, filter all the "executive" names from "exec_df" dataframe with the age in range "1950-01-01" to "1960-01-01". Ensure result is stored in a new datafram along with the executive names. Print the results in dataframe.


In [ ]:
## Your answer should go here

#### Task-2: Using SQL, filter all the "executive" names from "exec_df" dataframe with the age in range "1950-01-01" to "1960-01-01". Print the returned results. 